Live Streaming API Endpoint

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

app = FastAPI()

class LiveMatchCommentary(BaseModel):
    match_id: int
    commentary: str

def scrape_live_commentary(match_id: int) -> str:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-notifications")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(5)

    try:
        url = f"https://www.cricbuzz.com/live-cricket-scorecard/{match_id}"
        driver.get(url)
        time.sleep(2) 

        soup = BeautifulSoup(driver.page_source, "html.parser")

        commentary_block = soup.find("div", class_="cb-col cb-col-100 cb-com-ln")
        commentary = commentary_block.text.strip() if commentary_block else "No commentary found."

        return commentary
    finally:
        driver.quit()

@app.post("/get_live/{match_id}", response_model=LiveMatchCommentary)
def get_live_match_commentary(match_id: int):
    try:
        commentary = scrape_live_commentary(match_id)
        return {
            "match_id": match_id,
            "commentary": commentary
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error: {str(e)}")